In [63]:
import pandas as pd
import numpy as np

data1 = pd.read_csv("./input_data_1.csv",header=None).drop(0,axis=0).drop([0],axis=1)
data2 = pd.read_csv("./input_data_2.csv",header=None).drop(0,axis=0).drop([0],axis=1)

texts = np.concatenate([np.array(data1.iloc[2,:]),np.array(data2.iloc[2,:])])
labels = np.concatenate([np.array(data1.iloc[1,:]),np.array(data2.iloc[1,:])])
texts

array(['学生時代の友人7名で利用させて頂きました', 'ちょうど桜が咲いていて、別荘地内の桜もライトアップされていて、雰囲気良かったです',
       '信州安曇野、穂高温泉郷に位置する本格ログハウスの貸別荘です', 'コテージ内はとても清潔感があって安心して泊まれました',
       'インターから遠くもなく、コテージの近くにはコンビニや買い出しできるところもあるので、立地的にも満足でした',
       '趣味で一棟貸しのところをいろいろ利用していますがこちらの設備環境はとても清潔で周辺もとても静かなところで居心地が大変良かったです',
       '到着した日や翌日は雪の多い日でしたがスタッフの方が車や周辺の除雪をしてくれて大変助かりました',
       'また時間があれば利用させていただきたいと思います', 'ちょうど紅葉が綺麗な季節に滞在できました',
       'チェックインする前から、暖房がついていたりして、オーナーの気遣いを感じることができました',
       'ベッドも、羽毛布団で、快適に眠ることができました', '高速のインターからも程よい距離で、車で簡単に買い物も行けて良かったです',
       '今回は一泊二日だったので、次回は二泊くらいして、ゆっくり滞在したいです',
       '夜はとても静かで、部屋も涼しくて、快適で、家にいる時よりも安眠でグッスリ眠りました',
       '都会では目にすることができない綺麗な星空に家族全員感激していました',
       'どこにどんなスーパーがあり、銭湯があり、レストランがあるのか、という地図も分かりやすいです',
       '旅はいろんな場所に行きたいので同じ場所をリピートしたことがないのですが、ここは初めてまた行きたいと思えるところでした',
       'スタッフ皆さんはとても親切で優しい方ばかりなのがここの素敵なとこです',
       'コテージを一棟貸し切りで、お客様の別荘感覚でお使いいただけます。',
       '1月10日から一泊スキー旅行のお宿として、仲間数名で利用させていただきました',
       '予約する段階で、立地と薪ストーブに惹かれ即申込みました', '閑静な立地で、夜は星空がたいへんキレイでした',
    

In [64]:
import MeCab
import re

def tokenize(text):
    wakati = MeCab.Tagger("-O wakati")
    wakati.parse("")
    words = wakati.parse(text)

    # Make word list
    if words[-1] == u"\n":
        words = words[:-1]

    return words

In [65]:

texts = [tokenize(a) for a in texts]



In [66]:
from gensim.models import word2vec

model = word2vec.Word2Vec.load("./word2vec/model/wiki.model")


from Ocab import Ocab, Regexp
c = Regexp()
m = Ocab(target=["名詞","動詞","形容詞","副詞"])

In [67]:
# unknown_token = np.random.randint(-1, 1, (200, 1))  #
unknown_token = np.random.random_sample(200)
word2vec_array0 = []
for str in texts:
    word_vectors = []
    for char in m.removeStoplist(str, []).split(" "):
        try:
            word_vectors.append(model[char])
#             print('#{0}ok'.format(char))
        except KeyError:
#             print('#{0}vocabularry error'.format(char))
            for c in tokenize(char).split(" "):
                try:
                    word_vectors.append(model[c])
#                     print('#{0}ok'.format(c))
                except KeyError:
                    word_vectors.append(unknown_token)
#                     print('#{0}vocabularry error'.format(c))
    word2vec_array0.append(word_vectors)

len(word2vec_array0)

400

In [68]:
input_length = 25
input_data = []
for vector in word2vec_array0:
    context_vectors = vector
    while(len(context_vectors)<input_length):
        context_vectors = np.append(context_vectors,[np.full(200,-1)], axis=0)
#     print(context_vectors)
    input_data.append(context_vectors)
len(input_data) #400
len(input_data[0]) #25

25

In [69]:
from sklearn.model_selection import train_test_split
input_data = np.array(input_data)
output_data = np.array(output_data)
# train_input_data, val_input_data, train_output_data, val_output_data = train_test_split(input_data,output_data,train_size=0.8, test_size=0.2,random_state=42)


# print(input_data.shape)
# print(output_data.shape)
# print(len(data[2]))

In [76]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers import Dropout
from keras.layers.recurrent import LSTM
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras import backend as K 
from sklearn.model_selection import StratifiedKFold
from statistics import mean, median,variance,stdev
np.random.seed(0)





N_train = int(len(input_data) * 0.9)
N_validation = len(input_data) - N_train


X_train, X_validation, Y_train, Y_validation, row_train, row_validation = \
    train_test_split(input_data, labels, texts, test_size=N_validation)


maxlen = 25
'''
モデル設定
'''
n_in = 200  
n_hidden = 512
n_out = 1


def weight_variable(shape, name=None):
    return np.random.normal(scale=.01, size=shape)

def root_mean_squared_error(y_true, y_pred): 
     return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
# early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=1)






fold_num = 5 
seed = 5
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X_train, Y_train):
    model = Sequential()
    model.add(LSTM(n_hidden,
                   kernel_initializer=weight_variable,
                   input_shape=(maxlen, n_in)))
    # model.add(Activation('relu'))
    # model.add(Dropout(0.3))
    # model.add(Activation('relu'))
    # model.add(Dropout(0.5))
    model.add(Dense(n_out, kernel_initializer=weight_variable))


    model.summary()
    # optimizer = Adam(lr=0.0010, beta_1=0.9, beta_2=0.999)
    optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
    # model.compile(loss='mean_squared_error',optimizer=optimizer)
    model.compile(loss=root_mean_squared_error,
                  optimizer=optimizer)





    '''
    モデル学習
    '''
    epochs = 100
    batch_size = 256


    #earlystoppingなし
#     model.fit(X_train[train], Y_train[test],
#               batch_size=batch_size,
#               epochs=epochs,
#               validation_data=(X_validation, Y_validation))
    model.fit(X_train[train], Y_train[train],
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(X_validation, Y_validation))

    scores = model.evaluate(X_train[test], Y_train[test])
    print(scores)
    cvscores.append(scores)

print(cvscores)
print(mean(cvscores))

/Users/uchidayuki/Python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_40 (LSTM)               (None, 512)               1460224   
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 513       
Total params: 1,460,737
Trainable params: 1,460,737
Non-trainable params: 0
_________________________________________________________________
Train on 282 samples, validate on 40 samples
Epoch 1/100
282/282 [==============================] - 10s 35ms/step - loss: 3.9081 - val_loss: 0.6097
Epoch 2/100
282/282 [==============================] - 1s 4ms/step - loss: 0.5903 - val_loss: 0.6331
Epoch 3/100
282/282 [==============================] - 1s 4ms/step - loss: 0.5688 - val_loss: 0.6275
Epoch 4/100
282/282 [==============================] - 1s 4ms/step - loss: 0.5399 - val_loss: 0.5653
Epoch 5/100
282/282 [==============================] - 1s 4ms/step - loss: 0.

282/282 [==============================] - 2s 6ms/step - loss: 0.3036 - val_loss: 0.6671
Epoch 76/100
282/282 [==============================] - 2s 6ms/step - loss: 0.3875 - val_loss: 0.5500
Epoch 77/100
282/282 [==============================] - 2s 5ms/step - loss: 0.2807 - val_loss: 0.5261
Epoch 78/100
282/282 [==============================] - 2s 7ms/step - loss: 0.2691 - val_loss: 0.5371
Epoch 79/100
282/282 [==============================] - 2s 7ms/step - loss: 0.3993 - val_loss: 0.5559
Epoch 80/100
282/282 [==============================] - 2s 6ms/step - loss: 0.4102 - val_loss: 0.5415
Epoch 81/100
282/282 [==============================] - 2s 8ms/step - loss: 0.3597 - val_loss: 0.5666
Epoch 82/100
282/282 [==============================] - 2s 6ms/step - loss: 0.3839 - val_loss: 0.5196
Epoch 83/100
282/282 [==============================] - 2s 7ms/step - loss: 0.3166 - val_loss: 0.5285
Epoch 84/100
282/282 [==============================] - 2s 6ms/step - loss: 0.3500 - val_loss: 

286/286 [==============================] - 1s 4ms/step - loss: 0.4971 - val_loss: 0.4820
Epoch 49/100
286/286 [==============================] - 1s 4ms/step - loss: 0.3769 - val_loss: 0.5134
Epoch 50/100
286/286 [==============================] - 1s 4ms/step - loss: 0.3821 - val_loss: 0.5042
Epoch 51/100
286/286 [==============================] - 1s 4ms/step - loss: 0.4402 - val_loss: 0.5034
Epoch 52/100
286/286 [==============================] - 1s 4ms/step - loss: 0.3856 - val_loss: 0.4946
Epoch 53/100
286/286 [==============================] - 1s 4ms/step - loss: 0.4004 - val_loss: 0.4820
Epoch 54/100
286/286 [==============================] - 1s 4ms/step - loss: 0.3658 - val_loss: 0.4877
Epoch 55/100
286/286 [==============================] - 1s 5ms/step - loss: 0.3098 - val_loss: 0.5566
Epoch 56/100
286/286 [==============================] - 2s 6ms/step - loss: 0.4385 - val_loss: 0.4916
Epoch 57/100
286/286 [==============================] - 2s 6ms/step - loss: 0.3984 - val_loss: 

288/288 [==============================] - 1s 5ms/step - loss: 0.5450 - val_loss: 0.5907
Epoch 22/100
288/288 [==============================] - 2s 6ms/step - loss: 0.5029 - val_loss: 0.5898
Epoch 23/100
288/288 [==============================] - 2s 9ms/step - loss: 0.5557 - val_loss: 0.5829
Epoch 24/100
288/288 [==============================] - 2s 7ms/step - loss: 0.5028 - val_loss: 0.5775
Epoch 25/100
288/288 [==============================] - 2s 7ms/step - loss: 0.4986 - val_loss: 0.5756
Epoch 26/100
288/288 [==============================] - 2s 7ms/step - loss: 0.4980 - val_loss: 0.6013
Epoch 27/100
288/288 [==============================] - 2s 7ms/step - loss: 0.5309 - val_loss: 0.6234
Epoch 28/100
288/288 [==============================] - 2s 7ms/step - loss: 0.5518 - val_loss: 0.6612
Epoch 29/100
288/288 [==============================] - 2s 7ms/step - loss: 0.5885 - val_loss: 0.6106
Epoch 30/100
288/288 [==============================] - 2s 8ms/step - loss: 0.5375 - val_loss: 

72/72 [==============================] - 0s 3ms/step
0.5944781833224826
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_43 (LSTM)               (None, 512)               1460224   
_________________________________________________________________
dense_43 (Dense)             (None, 1)                 513       
Total params: 1,460,737
Trainable params: 1,460,737
Non-trainable params: 0
_________________________________________________________________
Train on 288 samples, validate on 40 samples
Epoch 1/100
288/288 [==============================] - 9s 32ms/step - loss: 3.8022 - val_loss: 0.8681
Epoch 2/100
288/288 [==============================] - 1s 4ms/step - loss: 0.8868 - val_loss: 0.5634
Epoch 3/100
288/288 [==============================] - 1s 4ms/step - loss: 0.5099 - val_loss: 0.6458
Epoch 4/100
288/288 [==============================] - 1s 4ms/step - loss: 0.5522 - val_loss: 0.5866
Epoch 

288/288 [==============================] - 2s 6ms/step - loss: 0.4047 - val_loss: 0.5132
Epoch 75/100
288/288 [==============================] - 2s 6ms/step - loss: 0.3464 - val_loss: 0.5076
Epoch 76/100
288/288 [==============================] - 2s 6ms/step - loss: 0.3912 - val_loss: 0.4976
Epoch 77/100
288/288 [==============================] - 2s 6ms/step - loss: 0.3023 - val_loss: 0.5350
Epoch 78/100
288/288 [==============================] - 2s 7ms/step - loss: 0.3730 - val_loss: 0.5245
Epoch 79/100
288/288 [==============================] - 2s 8ms/step - loss: 0.4187 - val_loss: 0.5041
Epoch 80/100
288/288 [==============================] - 2s 7ms/step - loss: 0.3210 - val_loss: 0.5330
Epoch 81/100
288/288 [==============================] - 2s 7ms/step - loss: 0.3075 - val_loss: 0.5081
Epoch 82/100
288/288 [==============================] - 2s 6ms/step - loss: 0.3382 - val_loss: 0.5222
Epoch 83/100
288/288 [==============================] - 2s 6ms/step - loss: 0.4252 - val_loss: 

296/296 [==============================] - 2s 8ms/step - loss: 0.6489 - val_loss: 0.5477
Epoch 48/100
296/296 [==============================] - 2s 8ms/step - loss: 0.4853 - val_loss: 0.5679
Epoch 49/100
296/296 [==============================] - 2s 7ms/step - loss: 0.4403 - val_loss: 0.6580
Epoch 50/100
296/296 [==============================] - 2s 6ms/step - loss: 0.5205 - val_loss: 0.5682
Epoch 51/100
296/296 [==============================] - 1s 4ms/step - loss: 0.4231 - val_loss: 0.5850
Epoch 52/100
296/296 [==============================] - 1s 4ms/step - loss: 0.4540 - val_loss: 0.6730
Epoch 53/100
296/296 [==============================] - 1s 4ms/step - loss: 0.5345 - val_loss: 0.5811
Epoch 54/100
296/296 [==============================] - 1s 4ms/step - loss: 0.4307 - val_loss: 0.5690
Epoch 55/100
296/296 [==============================] - 1s 5ms/step - loss: 0.4376 - val_loss: 0.6540
Epoch 56/100
296/296 [==============================] - 1s 4ms/step - loss: 0.5056 - val_loss: 

#### batch 32 epochs 100    mean   0.47
#### batch 64 epochs 100    mean   0.50
#### batch 128 epochs 100 mean 0.53  [0.5520341442181513, 0.5536023104513014, 0.5204473402765062, 0.45064101616541546, 0.55865877866745]
#### batch 256 epochs 100 mean 0525 [0.5234231841869843, 0.48426922108675985, 0.5944781833224826, 0.46878820326593185, 0.5575863420963287]



In [73]:
# test_vectors = np.array(test_vectors)
y_pred = model.predict(X_train)
y_test_pred = model.predict(X_validation)
# for text, pred in zip(row_train, y_train_pred):
#     print(text, pred)

for text, pred, ans in zip(row_validation, y_test_pred, Y_validation):
    print(text, pred, ans)

今回 は 品川 ブリンス 2 回 目 です が とても 満足 し て い ます  [3.2231724] 4.7
夕食 、 朝食 共 に 種類 が 豊富 で 、 美味しく 食べ すぎ て しまい まし た  [4.363337] 4.5
また 、 足湯 が あり 、 そこ から は 芦ノ湖 の 景色 が とても きれい に 見え まし た  [4.3757734] 4.5
お 部屋 は ダブル ベット を くっつけ て 利用 でき た ので 、 子ども が 落ちる かも 、 など の 心配 を せ ず ぐっすり 休め まし た  [4.3734064] 4.7
立地 が よく 、 電車 で 降り て から 荷物 を 預け て ユニバ に 行け た の が とても 便利 でし た  [4.4440517] 4.4
この 料金 で ここ まで の サービス を 受け られる の は お 得 だ と 思い ます  [3.3677924] 3.4
子供 が 小さい ので 、 早め に チェックイン し て 、 休憩  [3.5640833] 4.0
卓球 ・ カラオケ ・ 麻雀 ・ 囲碁 ・ ・ ・ 全て 無料 と は 驚き です  [3.0730922] 3.2
今回 は 忘れ物 を し て しまい 、 丁寧 に 取り計らっ て いただき まし た  [4.3757668] 4.5
お 風呂 は 窓 が あり 海 が 見え て 、 洗面 を 挟ん で シャワー 室 が あり 素敵 でし た  [4.0511274] 4.4
お 値段 は 少し 高く て も ハナ 館 で 宿泊 する 事 を 強く オススメ し ます  [4.0743694] 3.1
お 掃除 スタッフ の 方 が 海外 から の お客様 に 丁寧 に 対応 さ れ て い た の を み て 、 素敵 だ と 思い まし た  [4.3475437] 4.2
空調 等 設備 は 古い です が 、 問題 は なかっ た と 思い ます  [3.331078] 4.1
また 館内 用 の 浴衣 の デザイン も 素敵 で オシャレ だ な と 思い まし た  [4.068351] 4.0
今回 は 一 泊 二 日 だっ た ので 、 次回 は 二 泊 くらい し て 、 ゆっくり 滞在 し たい です  

In [75]:
'''
出力を用いて予測
'''
# 元データの最初の一部だけ切り出し
# model.predict()
# input_data[3:10].shape

test_text = [ "早く死んでくれ","バカだなぁ","昨日、バイトがあって、お客さんがたくさん来ました","浅井さんは丁寧で、非常に優しい","前にここに来たときは、大雪だったので、あまり楽しめませんでした","宿は長野県にある","この旅館は人気がある","従業員の方々は非常に優しく、対応が丁寧です","従業員の方々が優しかったです","佐々木希が来たらしい","料理がとっても美味しくて、最高でした！","非常に景色と温泉が綺麗で、部屋の内装やアメニティが良く、非常に満足出来るものでした"]
test_wakati_text = []
for text in test_text:
    test_wakati_text.append(tokenize(text))

# test_wakati_text = ""
word_model = word2vec.Word2Vec.load("./word2vec/model/wiki.model")
test_vectors = []
for wakati_text in test_wakati_text:
    test_vector = []
    for char in m.removeStoplist(wakati_text, []).split(" "):
        try:
            test_vector.append(word_model[char])
        except KeyError:
            test_vector.append(unknown_token)
    test_vectors.append(np.array(test_vector))

# test_vectors = np.array(test_vectors)
input_length = 25
input_test_data = []
for test_vector in test_vectors:
    context_vectors = test_vector
    while(len(context_vectors)<input_length):
        context_vectors = np.append(context_vectors,[np.full(200,-1)], axis=0)
    input_test_data.append(context_vectors)
input_test_data = np.array(input_test_data)


test_predictions = model.predict(input_test_data)

for pred, text in zip(test_predictions, test_text):
    print(text, pred)

早く死んでくれ [3.8762076]
バカだなぁ [3.2928483]
昨日、バイトがあって、お客さんがたくさん来ました [4.103437]
浅井さんは丁寧で、非常に優しい [3.8676064]
前にここに来たときは、大雪だったので、あまり楽しめませんでした [4.1435385]
宿は長野県にある [2.6897705]
この旅館は人気がある [3.7350123]
従業員の方々は非常に優しく、対応が丁寧です [4.15258]
従業員の方々が優しかったです [3.3799684]
佐々木希が来たらしい [3.4493246]
料理がとっても美味しくて、最高でした！ [4.270725]
非常に景色と温泉が綺麗で、部屋の内装やアメニティが良く、非常に満足出来るものでした [4.1179886]
